# Feature Selection using RFE and ANOVA
Previous automated feature selection using SKLearn's gridsearchCV was useful, but lacked some ability for interpretation and transparency. A traditional feature selection process is worth trying, as this will help reduce the number of variables needed for modeling and improve the interpretability of the final model.<br><br>
I plan on using recursive feature elimnition, which is an approximation of forward stepwise feature selection. Using Statmodels(), I plan on running an ANOVA on each feature against the outcome measure `death_within_7_days` to test for signficance. Each resulting F-test value will be modeled and added to an existing model containing other variables. This process starts with the most significant variable, adds the next significant variable, and continues on until limited improvements of explained variance are seen. This process is called *forward feature selection*.


In [1]:
#load libraries
import pandas as pd
import numpy as np
import statsmodels as sm
%matplotlib inline
import matplotlib.pyplot as plt

from IPython.display import display #displays full dataframe columns
#display all dataframe columns when printed
pd.options.display.max_columns = None

In [2]:
#load data
df = pd.read_csv('C:/Users/Mark.Burghart/Documents/projects/hospice_carepoint/data/transformed/carepoint_transformed_dummied.csv', index_col=0)
df.shape

(271541, 120)